In [1]:
%load_ext Cython

In [2]:
from collections import Counter, namedtuple
import numpy as np
from typing import Tuple, List, Optional, Sequence

In [ ]:
ar = np.sqrt(np.array([.5, 1, 2]))
scale = np.array([8, 16, 32])

In [ ]:
x_out, y_out = (32, 32)
y, x = np.meshgrid(np.arange(x_out), np.arange(y_out))
base_h, base_w = 32, 32

In [ ]:
x = x.reshape(-1, ).repeat(9)
y = y.reshape(-1, ).repeat(9)
x = x * base_w + base_w / 2
y = y * base_h + base_h / 2

In [ ]:
ar = np.tile(ar, x_out * y_out).repeat(3)
scale = np.tile(scale, x_out * y_out * 3)

In [ ]:
hs = np.round(base_h * scale * ar).astype(int)
ws = np.round(base_w * scale / ar).astype(int)

In [ ]:
boxes = np.vstack([x, y, ws, hs]).T

In [11]:
def generate_anchors(
    stride: int = 16,
    scales: List[int] = [8, 16, 32],
    ratios: List[int] = [1, 0.5, 2],
    base_dist: Optional[int] = None,
) -> np.ndarray:
    """computes anchor boxes for a given stride, scales and aspect ratios.
        Number of anchors = stride ** 2 * scales * aspect_ratio

    Keyword Arguments:
        stride {int} -- the subsampling ratio of the output feature map (default: {16})
        scales {List[int]} -- list of scales (default: {[8, 16, 32]})
        ratios {List[int]} -- list of aspect ratios (default: {[1, 0.5, 2]})

    Returns:
        [np.ndarray] -- shape: `(stride * stride * scales * ratios, 4)`
    """
    base_dist = base_dist if base_dist is not None else stride

    x_out, y_out = stride, stride
    base_h, base_w = base_dist, base_dist
    y, x = np.meshgrid(np.arange(x_out), np.arange(y_out))

    x = x.reshape(-1,).repeat(len(scales) * len(ratios))
    y = y.reshape(-1,).repeat(len(scales) * len(ratios))
    x = x * base_w + base_w / 2
    y = y * base_h + base_h / 2

    ratios = np.tile(np.array(ratios), x_out * y_out).repeat(3)
    scales = np.tile(np.array(scales), x_out * y_out * 3)

    hs = np.round(base_h * scales * ratios).astype(int)
    ws = np.round(base_w * scales / ratios).astype(int)

    return np.vstack([x, y, ws, hs]).T

def hw_to_minmax(bbox: np.ndarray, max_dim: Tuple[int, int]) -> np.ndarray:
    """given a set of bounding (anchor) boxes in the format `(x_center, y_center, width, height)`,
        transforms them to `(x_min, y_min, x_max, y_max)` format
    """
    boxes = np.zeros_like(bbox, dtype=np.int32)
    max_x, max_y = max_dim

    boxes[:, 0], boxes[:, 2] = (
        np.maximum(bbox[:, 0] - bbox[:, 2] / 2, 0).astype(int),
        np.minimum(bbox[:, 0] + bbox[:, 2] / 2, max_x).astype(int),
    )

    boxes[:, 1], boxes[:, 3] = (
        np.maximum(bbox[:, 1] - bbox[:, 3] / 2, 0).astype(int),
        np.minimum(bbox[:, 1] + bbox[:, 3] / 2, max_y).astype(int),
    )
    return boxes

In [44]:
%%cython

cimport cython
import numpy as np
cimport numpy as np

DTYPE = np.float32
ctypedef np.float32_t DTYPE_t
#ctypedef np

@cython.boundscheck(False)
@cython.wraparound(False)
def bbox_overlaps(
        np.ndarray[DTYPE_t, ndim=2] boxes,
        np.ndarray[DTYPE_t, ndim=2] query_boxes):
    """
    Parameters
    ----------
    boxes: (N, 4) ndarray of float
    query_boxes: (K, 4) ndarray of float
    Returns
    -------
    overlaps: (N, K) ndarray of overlap between boxes and query_boxes
    """
    cdef unsigned int N = boxes.shape[0]
    cdef unsigned int K = query_boxes.shape[0]
    cdef np.ndarray[DTYPE_t, ndim=2] overlaps = np.zeros((N, K), dtype=DTYPE)
    cdef DTYPE_t iw, ih, box_area
    cdef DTYPE_t ua
    cdef unsigned int k, n
    for k in range(K):
        box_area = (
            (query_boxes[k, 2] - query_boxes[k, 0] + 1) *
            (query_boxes[k, 3] - query_boxes[k, 1] + 1)
        )
        for n in range(N):
            iw = (
                min(boxes[n, 2], query_boxes[k, 2]) -
                max(boxes[n, 0], query_boxes[k, 0]) + 1
            )
            if iw > 0:
                ih = (
                    min(boxes[n, 3], query_boxes[k, 3]) -
                    max(boxes[n, 1], query_boxes[k, 1]) + 1
                )
                if ih > 0:
                    ua = float(
                        (boxes[n, 2] - boxes[n, 0] + 1) *
                        (boxes[n, 3] - boxes[n, 1] + 1) +
                        box_area - iw * ih
                    )
                    overlaps[n, k] = iw * ih / ua
    return overlaps

In [32]:
def bbox_iou(bbox_a, bbox_b):
    """Calculate the Intersection of Unions (IoUs) between bounding boxes.
    IoU is calculated as a ratio of area of the intersection
    and area of the union.
    This function accepts both :obj:`numpy.ndarray` and :obj:`cupy.ndarray` as
    inputs. Please note that both :obj:`bbox_a` and :obj:`bbox_b` need to be
    same type.
    The output is same type as the type of the inputs.
    Args:
        bbox_a (array): An array whose shape is :math:`(N, 4)`.
            :math:`N` is the number of bounding boxes.
            The dtype should be :obj:`numpy.float32`.
        bbox_b (array): An array similar to :obj:`bbox_a`,
            whose shape is :math:`(K, 4)`.
            The dtype should be :obj:`numpy.float32`.
    Returns:
        array:
        An array whose shape is :math:`(N, K)`. \
        An element at index :math:`(n, k)` contains IoUs between \
        :math:`n` th bounding box in :obj:`bbox_a` and :math:`k` th bounding \
        box in :obj:`bbox_b`.
    """
    if bbox_a.shape[1] != 4 or bbox_b.shape[1] != 4:
        raise IndexError

    # top left
    tl = np.maximum(bbox_a[:, None, :2], bbox_b[:, :2])
    # bottom right
    br = np.minimum(bbox_a[:, None, 2:], bbox_b[:, 2:])

    area_i = np.prod(br - tl, axis=2) * (tl < br).all(axis=2)
    area_a = np.prod(bbox_a[:, 2:] - bbox_a[:, :2], axis=1)
    area_b = np.prod(bbox_b[:, 2:] - bbox_b[:, :2], axis=1)
    return area_i / (area_a[:, None] + area_b - area_i)

In [28]:
def get_area(bbox):
    return (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])

def iou(box1, box2, eps = 1e-7):
    xmin = max(box1[0], box2[0])
    ymin = max(box1[1], box2[1])
    xmax = min(box1[2], box2[2])
    ymax = min(box1[3], box2[3])
    if (xmin >= xmax) or (ymin >= ymax):
        return 0
    intersection = (xmax - xmin) * (ymax - ymin)
    union = get_area(box1) + get_area(box2) - intersection
    return intersection / (union + eps)

def iou_vectorized(anchors, bbox, eps = 1e-7):
    xmin = np.maximum(anchors[:, 0], bbox[0])
    ymin = np.maximum(anchors[:, 1], bbox[1])
    xmax = np.minimum(anchors[:, 2], bbox[2])
    ymax = np.minimum(anchors[:, 3], bbox[3])
    
    intersection = np.maximum((xmax - xmin) * (ymax - ymin), 0.)
    intersection[np.where((xmin >= xmax) | (ymin >= ymax))[0]] = 0.
    union = (anchors[:, 2] - anchors[:, 0]) * (anchors[:, 3] - anchors[:, 1]) + get_area(bbox) - intersection
    return intersection / (union + 1e-7)

def make_labels(anchors, boxes, ignore = -1, img_dim = 512, n_classes = 1):
    minmax_anchors = hw_to_minmax(anchors, max_dim=(img_dim, img_dim)).astype(np.float32)
    class_ids, gt_boxes = [], []
    for box in boxes:
        class_ids.append(box.class_id)
        gt_boxes.append(list(box.bbox))
    
    
    return minmax_anchors, np.array(gt_boxes).astype(np.float32)
    #print(minmax_anchors.)
    #ious = bbox_overlaps(minmax_anchors, np.array(gt_boxes).astype(np.float32))
    #cls_gt = np.repeat(ignore, len(anchors) * n_classes).reshape(len(anchors), n_classes)
#     #cls_gt = []
#     for box in boxes: 
#         class_id = box.class_id
#         bbox = list(box.bbox)
#         ious = iou_vectorized(minmax_anchors, bbox)
#         cls_gt[np.where(ious > 0.7)[0], class_id - 1] = 1
        
#         if (cls_gt == ignore).all():
#             cls_gt[np.argmax(ious), class_id - 1] = 1
    
#         cls_gt[np.where(ious < 0.3)[0], class_id - 1] = 0

    return ious

In [8]:
Bbox = namedtuple('Bbox', 'xmin ymin xmax ymax')
Label = namedtuple('Label', ['class_id', 'bbox'])

In [9]:
lbl1 = Label(1, Bbox(0, 0, 140, 140))
lbl2 = Label(1, Bbox(20, 60, 200, 350))
lbl = [lbl1, lbl2]

In [12]:
anchors = generate_anchors(stride=32)

In [29]:
anchors, gt_boxes = make_labels(anchors, lbl, img_dim=1024, ignore=255)

In [41]:
%%time
iou1 = bbox_overlaps(anchors, gt_boxes)

CPU times: user 202 µs, sys: 12 µs, total: 214 µs
Wall time: 206 µs


In [36]:
%%time
iou2 = bbox_iou(anchors, gt_boxes)

CPU times: user 1.1 ms, sys: 921 µs, total: 2.03 ms
Wall time: 1.11 ms


In [39]:
max_iou = iou1.max(axis=1)